In [9]:
# This worked with python 3, with requests module installed
# use port 8443 for production, 8444 for test
# register at https://www.nsgportal.org/reg/reg.php for username and password
CRA_USER = 'aacwb'
PASSWORD = 'Manu@#1620'
# for development version:
# log in at https://nsgr.sdsc.edu:8444/restusers/login.action
# for production version:
# log in at https://nsgr.sdsc.edu:8443/restusers/login.action
# Tool names can be found at Developer->Documentation (Tools: How to Configure Specific Tools)
# create a new application at Developer->Application Management (Create New Application)
# save the Application Key for use in REST requests
KEY = 'MyFirstApp-911B7CAD877846678763B2014752F2C0'
# for development version:
# URL = 'https://nsgr.sdsc.edu:8444/cipresrest/v1'
# for production version:
URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1'
TOOL = 'NEURON73_TG'


In [10]:
%%capture --no-stderr submitoutput

import os
import requests
import xml.etree.ElementTree
import time
import sys

os.popen('pwd').read()
os.popen('ls -ltr').read()
headers = {'cipres-appkey' : KEY}
payload = {'tool' : TOOL, 'metadata.statusEmail' : 'true', 'vparam.number_cores_' : 24, 'vparam.number_nodes_' : 2, 'vparam.runtime_' : 0.5, 'vparam.filename_': 'Batch.hoc', 'vparam.cmdlineopts_': '-c TSTOP=1'}
files = {'input.infile_' : open(r'C:\Users\Radhakrishna\CyNeuroCloud\CyNeuroLaravel\public\CyNeuroSimpleWorkflowExample.zip','rb')}

r = requests.post('{}/job/{}'.format(URL, CRA_USER), auth=(CRA_USER, PASSWORD), data=payload, headers=headers, files=files)
#print(r.text)

root = xml.etree.ElementTree.fromstring(r.text)

sys.stderr.write("%s\n" % r.text)
sys.stderr.write("%s\n" % r.url)

for child in root:
    if child.tag == 'resultsUri':
        for urlchild in child:
            if urlchild.tag == 'url':
                outputuri = urlchild.text
    if child.tag == 'selfUri':
        for urlchild in child:
            if urlchild.tag == 'url':
                selfuri = urlchild.text
#print(outputuri,file=sys.stderr)
sys.stderr.write("%s\n" % outputuri)
#print(selfuri,file=sys.stderr)
sys.stderr.write("%s\n" % selfuri)

headers = {'cipres-appkey' : KEY}

#print('Waiting for job to complete',file=sys.stderr)
sys.stderr.write('Waiting for job to complete\n')
jobdone = 0
while jobdone == 0:
    r = requests.get(selfuri, auth=(CRA_USER, PASSWORD), headers=headers)
    #print(r.text)
    root = xml.etree.ElementTree.fromstring(r.text)
    for child in root:
        if child.tag == 'terminalStage':
            jobstatus = child.text
            if jobstatus == 'false':
                time.sleep(5)
                #print('.',file=sys.stderr,end='')
                sys.stderr.write('.')
            else:
                jobdone = 1
                #print('',file=sys.stderr,end='\n')
                sys.stderr.write('\n')
                break

#print('Job completion detected, getting download URIs...',file=sys.stderr)
sys.stderr.write('Job completion detected, getting download URIs...')
r = requests.get(outputuri,
                 headers= headers, auth=(CRA_USER, PASSWORD))
#print(r.text)
globaldownloadurilist = []
root = xml.etree.ElementTree.fromstring(r.text)
for child in root:
    if child.tag == 'jobfiles':
        for jobchild in child:
            if jobchild.tag == 'jobfile':
                for downloadchild in jobchild:
                    if downloadchild.tag == 'downloadUri':
                        for attchild in downloadchild:
                            if attchild.tag == 'url':
                                #print(attchild.text)
                                sys.stdout.write(attchild.text)
                                globaldownloadurilist.append(attchild.text)
                
#print('Download complete.  Run the next cell.',file=sys.stderr)
sys.stderr.write('Download complete.  Run the next cell.\n')

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<jobstatus>
    <selfUri>
        <url>https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-8995E9765E3D43378A2E5BD3A8BD54B1</url>
        <rel>jobstatus</rel>
        <title>NGBW-JOB-NEURON73_TG-8995E9765E3D43378A2E5BD3A8BD54B1</title>
    </selfUri>
    <jobHandle>NGBW-JOB-NEURON73_TG-8995E9765E3D43378A2E5BD3A8BD54B1</jobHandle>
    <jobStage>QUEUE</jobStage>
    <terminalStage>false</terminalStage>
    <failed>false</failed>
    <metadata>
        <entry>
            <key>statusEmail</key>
            <value>true</value>
        </entry>
    </metadata>
    <dateSubmitted>2018-09-17T15:50:13-07:00</dateSubmitted>
    <resultsUri>
        <url>https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-8995E9765E3D43378A2E5BD3A8BD54B1/output</url>
        <rel>results</rel>
        <title>Job Results</title>
    </resultsUri>
    <workingDirUri>
        <url>https://nsgr.sdsc.edu:8443/cipresrest/v1

In [11]:
#submitoutput.show()
#print(submitoutput.stdout)
#print(globaldownloadurilist)
globaloutputdict = {}
for downloaduri in globaldownloadurilist:
    r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers)
    #print(r.text)
    globaloutputdict[downloaduri] = r.text
for name in globaloutputdict.keys():
    #continue_var = raw_input("display %s [Y/y]? " % name)
    if sys.version_info[0] < 3:
        continue_var = raw_input("display %s [Y/y]?" % name)
    else:
        continue_var = input("display %s [Y/y]?" % name)
    if continue_var in ['Y','y']:
         #print(globaloutputdict[name])
         sys.stdout.write("%s\n" % globaloutputdict[name])   

display https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-8995E9765E3D43378A2E5BD3A8BD54B1/output/27223 [Y/y]?n
display https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-8995E9765E3D43378A2E5BD3A8BD54B1/output/27224 [Y/y]?n
display https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-8995E9765E3D43378A2E5BD3A8BD54B1/output/27225 [Y/y]?n


In [12]:
#http://stackoverflow.com/questions/31804799/how-to-get-pdf-filename-with-python-requests
#downloaduri = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/kenneth/NGBW-JOB-NEURON73_TG-650AAA3A8044475580739C88BDF7771D/output/14'
for downloaduri in globaldownloadurilist:
    r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers)
    sys.stderr.write("%s\n" % r.headers)
    import re
    d = r.headers['content-disposition']
    fname_list = re.findall("filename=(.+)", d)
    for fname in fname_list:
        sys.stderr.write("%s\n" % fname)

{'Server': 'Apache-Coyote/1.1', 'Cache-Control': 'private', 'Expires': 'Wed, 31 Dec 1969 16:00:00 PST', 'Content-Disposition': 'attachment; filename=STDOUT', 'Content-Type': 'text/plain', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 17 Sep 2018 22:53:19 GMT'}
STDOUT
{'Server': 'Apache-Coyote/1.1', 'Cache-Control': 'private', 'Expires': 'Wed, 31 Dec 1969 16:00:00 PST', 'Content-Disposition': 'attachment; filename=STDERR', 'Content-Type': 'text/plain', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 17 Sep 2018 22:53:19 GMT'}
STDERR
{'Server': 'Apache-Coyote/1.1', 'Cache-Control': 'private', 'Expires': 'Wed, 31 Dec 1969 16:00:00 PST', 'Content-Disposition': 'attachment; filename=output.tar.gz', 'Content-Type': 'text/plain', 'Transfer-Encoding': 'chunked', 'Date': 'Mon, 17 Sep 2018 22:53:19 GMT'}
output.tar.gz


In [13]:
# download all output files

import re
#downloaduri = 'https://cipresrest.sdsc.edu/cipresrest/v1/job/kenneth/NGBW-JOB-RAXMLHPC8_REST_XSEDE-EF0E26B61D2E4C0C8D02499D8068D873/output/11496'
for downloaduri in globaldownloadurilist:
    r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers)
    #sys.stderr.write("%s\n" % r.json())
    #r.content
    d = r.headers['content-disposition']
    filename_list = re.findall('filename=(.+)', d)
    for filename in filename_list:
        #http://docs.python-requests.org/en/master/user/quickstart/#raw-response-content
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content():
                fd.write(chunk)

In [14]:
# get a list of jobs for user and app key, and terminalStage status

r = requests.get("%s/job/%s" % (URL,CRA_USER), auth=(CRA_USER, PASSWORD), headers=headers)
#print(r.text)

root = xml.etree.ElementTree.fromstring(r.text)
for child in root:
    if child.tag == 'jobs':
        for jobchild in child:
            if jobchild.tag == 'jobstatus':
                for statuschild in jobchild:
                    if statuschild.tag == 'selfUri':
                        for selfchild in statuschild:
                            if selfchild.tag == 'url':
                                #print(child)
                                joburi = selfchild.text
                                jobr = requests.get(joburi, auth=(CRA_USER, PASSWORD), headers=headers)
                                jobroot = xml.etree.ElementTree.fromstring(jobr.text)
                                for jobrchild in jobroot:
                                    if jobrchild.tag == 'terminalStage':
                                        jobstatus = jobrchild.text
                                        sys.stdout.write("job url: %s status terminalStage: %s\n" % (joburi,jobstatus))

job url: https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-30BF952B5E5C483E92380A7A34FC7013 status terminalStage: true
job url: https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-8995E9765E3D43378A2E5BD3A8BD54B1 status terminalStage: true


In [15]:
# get information for a single job, print out raw XML, need to set joburi according to above list

joburi = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/kenneth/NGBW-JOB-NEURON73_TG-220F7B3C7EE84BC3ADD87346E933ED5E'
r = requests.get(joburi,
                 headers= headers, auth=(CRA_USER, PASSWORD))
print(r.text)


<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<error>
    <displayMessage>Authorization Error</displayMessage>
    <message>org.ngbw.cipresrest.auth.AuthorizationException: Access forbidden for aacwb.
</message>
    <code>1</code>
</error>



In [8]:
# delete an old job, need to set joburi
joburi = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/kenneth/NGBW-JOB-NEURON73_TG-7AF604008F314B43A331231E5A94C361'
r = requests.delete(joburi, auth=(CRA_USER, PASSWORD), headers=headers)
sys.stderr.write("%s\n" % r.text)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<error>
    <displayMessage>Authorization Error</displayMessage>
    <message>org.ngbw.cipresrest.auth.AuthorizationException: Access forbidden for aacwb.
</message>
    <code>1</code>
</error>

